In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
%cd '/mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/mcmaster-125_3378_all_pdfs'

/mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/mcmaster-125_3378_all_pdfs


In [2]:
import json
import os
from pathlib import Path
from pickle import FALSE
import re
import shutil

In [24]:
import os
import shutil
import json

class DataReader:
    def __init__(self, data_dir, encoding='utf-8'):
        self.data_dir = data_dir
        self.encoding = encoding

    def read_text(self, filename):
        path = os.path.join(self.data_dir, filename)
        with open(path, 'r', encoding=self.encoding) as f:
            return f.read()

    def read_lines(self, filename):
        path = os.path.join(self.data_dir, filename)
        if os.path.exists(path):
            with open(path, 'r', encoding=self.encoding) as f:
                return f.readlines()
        else:
            print(f"File does not exist: {path}")
            return []

    def read_json(self, filename):
        path = os.path.join(self.data_dir, filename + '.json')
        with open(path, 'r', encoding=self.encoding) as f:
            return json.load(f)

    def read_json_from_path(self, path):
        with open(path, 'r', encoding=self.encoding) as f:
            return json.load(f)

class DataWriter:
    def __init__(self, data_dir, encoding='utf-8'):
        self.data_dir = data_dir
        self.encoding = encoding

    def write_text(self, filename, text):
        path = os.path.join(self.data_dir, filename)
        with open(path, 'a', encoding=self.encoding) as f:
            f.write(text)

    def write_json(self, filename, data):
        path = os.path.join(self.data_dir, filename + '.json')
        with open(path, 'w', encoding=self.encoding) as f:
            json.dump(data, f)

    def write_json_from_path(self, path, data):
        with open(path, 'w', encoding=self.encoding) as f:
            json.dump(data, f)

In [29]:
if __name__ == '__main__':
    root_dir = '/mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/'
    data_dir = os.path.join(root_dir, 'Paper_Supporting_Test/')
    target_dir = os.path.join(root_dir, 'Paper_Supporting_Test_pp/')

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print("Created target directory:", target_dir)

    reader = DataReader(data_dir)
    writer = DataWriter(target_dir)

    text_files = [f for f in os.listdir(data_dir) if f.endswith('_0.txt')]
    print("Files to process:", len(text_files))

    with open(os.path.join(target_dir, 'Label.txt'), 'a') as l:
        for f in text_files:
            try:
                image_path = f.replace('_0.txt', '_ori.jpg')
                image_src = os.path.join(data_dir, image_path)
                image_target = os.path.join(target_dir, image_path)

                if not os.path.exists(image_target):
                    shutil.copy(image_src, image_target)
                    print("Copied image to target:", image_target)

                l.write(target_dir.split('/')[-2] + '/' + image_path + '\t')
                print('fileState: ' + target_dir.split('/')[-2] + '/' + image_path + '\t')
                writer.write_text('fileState.txt', target_dir.split('/')[-2] + '/' + image_path + '\t1\n')

                data_line = reader.read_lines(f)
                if not data_line:
                    print("No data read from file:", f)
                l.write('[')
                for line in data_line:
                    words = line.split('\t')
                    if len(words) < 5:
                        print("Skipping malformed line:", line)
                        continue
                    word = words[0]
                    points = list(map(int, map(float, words[1:5])))
                    label_write = {
                        'transcription': word,
                        'points': [[points[0], points[1]], [points[2], points[1]], [points[2], points[3]], [points[0], points[3]]],
                        'difficult': False,
                        'key_cls': ''
                    }
                    json.dump(label_write, l, ensure_ascii=False)
                    if line != data_line[-1]:
                        l.write(', ')
                l.write(']')
            except Exception as e:
                print(f"An error occurred while processing file {f}: {str(e)}")
            l.write('\n')

Files to process: 0
